# Mastering selection and filtering of data frames

We will work with a data set from [Kleinteich and Gorb, *Sci. Rep.*, **4**, 5355, 2014](https://doi.org/10.1038/srep05225), and was [featured in the New York Times](http://www.nytimes.com/2014/08/25/science/a-frog-thats-a-living-breathing-pac-man.html). They measured several properties about the tongue strikes of horned frogs. Let's take a look at the data set, which is in the file `../data/frog_tongue_adhesion.csv`.

In [2]:
data_path = '../data/'
!head -20 ../data/frog_tongue_adhesion.csv

# These data are from the paper,
#   Kleinteich and Gorb, Sci. Rep., 4, 5225, 2014.
# It was featured in the New York Times.
#    http://www.nytimes.com/2014/08/25/science/a-frog-thats-a-living-breathing-pac-man.html
#
# The authors included the data in their supplemental information.
#
# Importantly, the ID refers to the identifites of the frogs they tested.
#   I:   adult, 63 mm snout-vent-length (SVL) and 63.1 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   II:  adult, 70 mm SVL and 72.7 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   III: juvenile, 28 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
#   IV:  juvenile, 31 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (m

## Tasks
The first lines all begin with `#` signs, signifying that they are comments and not data. They do give important information, though, such as the meaning of the ID data. The ID refers to which specific frog was tested.

Immediately after the comments, we have a row of comma-separated headers. This row sets the number of columns in this data set and labels the meaning of the columns. So, we see that the first column is the date of the experiment, the second column is the ID of the frog, the third is the trial number, and so on.

After this row, each row represents a single experiment where the frog struck the target. So, these data are already in tidy format. 

**a)** Load in the data set into a data frame. Be sure to use the appropriate value for the `comment_prefix` keyword argument of `pl.read_csv()`.

**b)** Extract the impact time of all impacts that had an adhesive strength of magnitude greater than 2000 Pa. *Note*: The data in the `'adhesive strength (Pa)'` column is all negative. This is because the adhesive force is defined to be negative in the measurement. Without changing the data in the data frame, how can you check that the magnitude (the absolute value) is greater than 2000?

**c)** Extract the impact force and adhesive force for all of Frog II's strikes.

**d)** Extract the adhesive force and the time the frog pulls on the target for juvenile frogs (Frogs III and IV). *Hint*: We saw the `&` operator for Boolean indexing across more than one column. The `|` operator signifies OR, and works analogously. For technical reasons that we can discuss if you like, the Python operators `and` and `or` will **not** work for Boolean indexing of data frames. You could also approach this using the `is_in()` method of a Polars `Expression`.

In [3]:
# a) Load in the data set into a data frame. 
# Be sure to use the appropriate value for the `comment_prefix` keyword argument of `pl.read_csv()`.
import polars as pl
import os
fname = os.path.join(data_path, "frog_tongue_adhesion.csv")
df = pl.read_csv(fname, comment_prefix="#")
df.head()

date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa)
str,str,i64,i64,i64,f64,i64,i64,f64,f64,i64,i64,f64,i64,i64
"""2013_02_26""","""I""",3,1205,46,1.95,-785,884,1.27,-0.29,387,70,0.82,3117,-2030
"""2013_02_26""","""I""",4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695
"""2013_03_01""","""I""",1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239
"""2013_03_01""","""I""",2,1556,41,2.51,-455,1025,0.74,-0.17,330,158,0.52,4718,-1381
"""2013_03_01""","""I""",3,493,36,0.8,-974,499,1.57,-0.423,245,216,0.12,2012,-3975


In [4]:
# b) Extract the impact time of all impacts that had an adhesive strength of magnitude greater than 2000 Pa. 
# *Note*: The data in the `'adhesive strength (Pa)'` column is all negative. 
# This is because the adhesive force is defined to be negative in the measurement. 
# Without changing the data in the data frame, how can you check that the magnitude (the absolute value) is greater than 2000?
filtered_imapct_times = df.filter(pl.col('adhesive strength (Pa)').abs()>2000).get_column('impact time (ms)')
filtered_imapct_times

impact time (ms)
i64
46
44
34
36
46
…
6
31
34


In [5]:
# c) Extract the impact force and adhesive force for all of Frog II's strikes.
# ID = "II", 
required_id = "II"
imp_force_col = "impact force (mN)"
adh_force_col = "adhesive force (mN)"
selected_df_c = df.filter((pl.col("ID")==required_id)).select([imp_force_col,adh_force_col])
selected_df_c

impact force (mN),adhesive force (mN)
i64,i64
1612,-655
605,-292
327,-246
946,-245
541,-553
…,…
515,-599
435,-364
383,-469


In [6]:
# d) Extract the adhesive force and the time the frog pulls on the target for juvenile frogs (Frogs III and IV).
# *Hint*: We saw the `&` operator for Boolean indexing across more than one column. 
# The `|` operator signifies OR, and works analogously. 
# For technical reasons that we can discuss if you like, the Python operators `and` and `or` will **not** work for Boolean indexing of data frames. 
# You could also approach this using the `is_in()` method of a Polars `Expression`.
col1 = "adhesive force (mN)"
col2 = "time frog pulls on target (ms)"
selected_df_d = df.filter(pl.col('ID').is_in(['III','IV'])).select([col1, col2])
selected_df_d

adhesive force (mN),time frog pulls on target (ms)
i64,i64
-94,683
-163,245
-172,619
-225,1823
-301,918
…,…
-302,986
-216,1627
-163,2021


## Excercise 2
We will continue working with the frog tongue adhesion data set.

You’ll now practice your split-apply-combine skills. First load in the data set. Then,

a) Compute standard deviation of the impact forces for each frog.

b) Compute the coefficient of variation of the impact forces and adhesive forces for each frog.

c) Compute a data frame that has the mean, median, standard deviation, and coefficient of variation of the impact forces and adhesive forces for each frog.

In [7]:
df.head()

date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa)
str,str,i64,i64,i64,f64,i64,i64,f64,f64,i64,i64,f64,i64,i64
"""2013_02_26""","""I""",3,1205,46,1.95,-785,884,1.27,-0.29,387,70,0.82,3117,-2030
"""2013_02_26""","""I""",4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695
"""2013_03_01""","""I""",1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239
"""2013_03_01""","""I""",2,1556,41,2.51,-455,1025,0.74,-0.17,330,158,0.52,4718,-1381
"""2013_03_01""","""I""",3,493,36,0.8,-974,499,1.57,-0.423,245,216,0.12,2012,-3975


In [8]:
# a) Compute standard deviation of the impact forces for each frog.
df.group_by('ID').agg(pl.col('impact force (mN)').std().alias('impact_force_std_by_id'))

ID,impact_force_std_by_id
str,f64
"""II""",424.573256
"""IV""",234.864328
"""III""",124.273849
"""I""",630.207952


In [9]:
# b) Compute the coefficient of variation of the impact forces and adhesive forces for each frog.
cov = lambda col: col.std() / col.mean()
df.group_by('ID').agg([cov(pl.col('impact force (mN)')).alias('impact_force_cov_by_id'),
                      cov(pl.col('adhesive force (mN)').alias('adhesive_force_cov_by_id'))])

ID,impact_force_cov_by_id,adhesive_force_cov_by_id
str,f64,f64
"""III""",0.225911,-0.426227
"""I""",0.411847,-0.253863
"""II""",0.600231,-0.440864
"""IV""",0.560402,-0.316045


In [10]:
# c) Compute a data frame that has the mean, median, standard deviation, and coefficient of variation of the impact forces and adhesive forces for each frog.
def return_stats(col):
    returned_list = [
        pl.col(col).mean().alias(f"{col} - mean"),
        pl.col(col).median().alias(f"{col} - median"),
        pl.col(col).std().alias(f"{col} - std"),
        (pl.col(col).std()/pl.col(col).mean()).alias(f"{col} - cov")
    ]
    return returned_list

df.group_by('ID').agg(return_stats('impact force (mN)') + return_stats('adhesive force (mN)'))

ID,impact force (mN) - mean,impact force (mN) - median,impact force (mN) - std,impact force (mN) - cov,adhesive force (mN) - mean,adhesive force (mN) - median,adhesive force (mN) - std,adhesive force (mN) - cov
str,f64,f64,f64,f64,f64,f64,f64,f64
"""II""",707.35,573.0,424.573256,0.600231,-462.3,-517.0,203.8116,-0.440864
"""III""",550.1,544.0,124.273849,0.225911,-206.75,-201.5,88.122448,-0.426227
"""I""",1530.2,1550.5,630.207952,0.411847,-658.4,-664.5,167.143619,-0.253863
"""IV""",419.1,460.5,234.864328,0.560402,-263.6,-233.5,83.309442,-0.316045


## Excercise 3
So, each frog has associated with it an age (adult or juvenile), snout-vent-length (SVL), body weight, and species (either cross or cranwelli). For a tidy data frame, we should have a column for each of these values. Your task is to load in the data, and then add these columns to the data frame. For convenience, here is a data frame with data about each frog.
```
#   I:   adult, 63 mm snout-vent-length (SVL) and 63.1 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   II:  adult, 70 mm SVL and 72.7 g body weight,
#        Ceratophrys cranwelli crossed with Ceratophrys cornuta
#   III: juvenile, 28 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
#   IV:  juvenile, 31 mm SVL and 12.7 g body weight, Ceratophrys cranwelli
```

In [11]:
df_frog_metadata = pl.DataFrame(
    data={
        "ID": ["I", "II", "III", "IV"],
        "age": ["adult", "adult", "juvenile", "juvenile"],
        "SVL (mm)": [63, 70, 28, 31],
        "weight (g)": [63.1, 72.7, 12.7, 12.7],
        "species": ["cross", "cross", "cranwelli", "cranwelli"],
    }
)

In [12]:
joined_df = df.join(df_frog_metadata, on='ID', how='full',coalesce=True)
joined_df.head()

date,ID,trial number,impact force (mN),impact time (ms),impact force / body weight,adhesive force (mN),time frog pulls on target (ms),adhesive force / body weight,adhesive impulse (N-s),total contact area (mm2),contact area without mucus (mm2),contact area with mucus / contact area without mucus,contact pressure (Pa),adhesive strength (Pa),age,SVL (mm),weight (g),species
str,str,i64,i64,i64,f64,i64,i64,f64,f64,i64,i64,f64,i64,i64,str,i64,f64,str
"""2013_02_26""","""I""",3,1205,46,1.95,-785,884,1.27,-0.29,387,70,0.82,3117,-2030,"""adult""",63,63.1,"""cross"""
"""2013_02_26""","""I""",4,2527,44,4.08,-983,248,1.59,-0.181,101,94,0.07,24923,-9695,"""adult""",63,63.1,"""cross"""
"""2013_03_01""","""I""",1,1745,34,2.82,-850,211,1.37,-0.157,83,79,0.05,21020,-10239,"""adult""",63,63.1,"""cross"""
"""2013_03_01""","""I""",2,1556,41,2.51,-455,1025,0.74,-0.17,330,158,0.52,4718,-1381,"""adult""",63,63.1,"""cross"""
"""2013_03_01""","""I""",3,493,36,0.8,-974,499,1.57,-0.423,245,216,0.12,2012,-3975,"""adult""",63,63.1,"""cross"""


# Plotting in class

In [14]:
import iqplot
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [23]:
bokeh.io.show(
    iqplot.stripbox(
        data=df,
        q="impact force (mN)",
        cats =['ID'],
        spread='jitter', # 'jitter', 'swarm'
    )
)